In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
df = spark.sql("SELECT * FROM LakhehouseTF_DataEngineering.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a08d930f-879e-4b4e-a107-1da68fe89d31)

In [2]:
display(df.select("snippet"))

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1339d6d8-7075-4d12-acc4-7b24ab09157a)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

model = (AnalyzeText()
        .setTextCol("snippet")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("Error"))

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 5, Finished, Available, Finished)

In [4]:
result = model.transform(df)

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c33f75f4-ca51-4daf-9e85-c20ce664e7ea)

In [6]:
from pyspark.sql.functions import col
sentiment_df = result.withColumn("Sentiment",col("response.documents.sentiment"))

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 8, Finished, Available, Finished)

In [7]:
df_final = sentiment_df.drop("error","response")

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 9, Finished, Available, Finished)

In [8]:
display(df_final)

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3fef8e59-1a04-4796-ad4e-50f166e8f076)

In [9]:
from pyspark.sql.utils import AnalysisException
 
table_name = "LakhehouseTF_DataEngineering.tbl_sentiment_analysis"
 
spark.sql("CREATE DATABASE IF NOT EXISTS LakhehouseTF_DataEngineering")
 
try:
    # Try creating the table fresh
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} created successfully.")
 
except AnalysisException:
    print(f"Table {table_name} already exists. Dropping and recreating...")
 
    # Drop the existing table
    spark.sql(f"DROP TABLE IF EXISTS {table_name}")
 
    # Recreate with df_final
    df_final.write.format("delta").saveAsTable(table_name)
    print(f"Table {table_name} recreated successfully.")

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 11, Finished, Available, Finished)

Table LakhehouseTF_DataEngineering.tbl_sentiment_analysis created successfully.


In [11]:
%%sql
select * from tbl_sentiment_analysis

StatementMeta(, 43e49317-5de8-448a-81f1-842a44314258, 12, Finished, Available, Finished)

<Spark SQL result set with 44 rows and 14 fields>